In [ ]:
%load_ext autoreload
%autoreload 2
%load_ext line_profiler

In [ ]:
import transformers, nltk, pandas as pd, torch, string
from datasets import load_dataset, load_from_disk, DatasetDict, ClassLabel
from pprint import pprint
from datetime import datetime
import argparse
import functools


from textattack import Attack, AttackArgs,Attacker
from textattack.models.wrappers import HuggingFaceModelWrapper
from textattack.datasets import HuggingFaceDataset
from textattack.loggers import CSVLogger # tracks a dataframe for us.
from textattack.attack_recipes import AttackRecipe
from textattack.search_methods import BeamSearch
from textattack.constraints import Constraint
from textattack.constraints.pre_transformation import RepeatModification, StopwordModification
from textattack.transformations import WordSwapEmbedding, WordSwapMaskedLM
from textattack.goal_functions import UntargetedClassification
from textattack.metrics.attack_metrics.attack_success_rate import AttackSuccessRate
from textattack.metrics.attack_metrics.words_perturbed import WordsPerturbed
from textattack.metrics.attack_metrics.attack_queries import AttackQueries
from textattack.metrics.quality_metrics.perplexity import Perplexity
from textattack.metrics.quality_metrics.use import USEMetric
from sentence_transformers.util import pytorch_cos_sim

from travis_attack.utils import display_all, merge_dicts, append_df_to_csv, set_seed
from travis_attack.data import prep_dsd_rotten_tomatoes,prep_dsd_simple,prep_dsd_financial
from travis_attack.config import Config
from travis_attack.models import _prepare_vm_tokenizer_and_model, get_vm_probs, prepare_models, get_nli_probs
from travis_attack.baseline_attacks import AttackRecipes, setup_baselines_parser
from fastcore.basics import in_jupyter


import warnings
warnings.filterwarnings("ignore", message="FutureWarning: The frame.append method is deprecated") 

path_baselines = "./baselines/"

set_seed(1000)


In [ ]:
!jupyter nbconvert \
    --TagRemovePreprocessor.enabled=True \
    --TagRemovePreprocessor.remove_cell_tags="['hide']" \
    --TemplateExporter.exclude_markdown=True \
    --to python "baselines.ipynb"

[NbConvertApp] Converting notebook baselines.ipynb to python


In [ ]:
######### CONFIG (default values) #########
param_d = dict(
    ds_name = "financial",
    split='test',
    sts_threshold = 0.8,
    contradiction_threshold = 0.2,
    acceptability_threshold = 0.5,
    pp_letter_diff_threshold = 30
)
###########################################

if not in_jupyter():  # override with any script options
    parser = setup_baselines_parser()
    newargs = vars(parser.parse_args())
    for k,v in newargs.items(): 
        if v is not None: param_d[k] = v

In [ ]:
### Common attack components
attack_recipes = AttackRecipes(param_d)
attack_list = attack_recipes.get_attack_list()

Reusing dataset financial_phrasebank (/data/tproth/.cache/huggingface/datasets/financial_phrasebank/sentences_50agree/1.0.0/a6d468761d4e0c8ae215c77367e1092bead39deb08fbf4bffd7c0a6991febbf0)


Loading cached split indices for dataset at /data/tproth/.cache/huggingface/datasets/financial_phrasebank/sentences_50agree/1.0.0/a6d468761d4e0c8ae215c77367e1092bead39deb08fbf4bffd7c0a6991febbf0/cache-8cd79ddb74449b16.arrow and /data/tproth/.cache/huggingface/datasets/financial_phrasebank/sentences_50agree/1.0.0/a6d468761d4e0c8ae215c77367e1092bead39deb08fbf4bffd7c0a6991febbf0/cache-767acf0081605736.arrow
Loading cached split indices for dataset at /data/tproth/.cache/huggingface/datasets/financial_phrasebank/sentences_50agree/1.0.0/a6d468761d4e0c8ae215c77367e1092bead39deb08fbf4bffd7c0a6991febbf0/cache-8bc5c0fec38cf232.arrow and /data/tproth/.cache/huggingface/datasets/financial_phrasebank/sentences_50agree/1.0.0/a6d468761d4e0c8ae215c77367e1092bead39deb08fbf4bffd7c0a6991febbf0/cache-ea0232ccd720d2dc.arrow


textattack: Unknown if model of class <class 'transformers.models.roberta.modeling_roberta.RobertaForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


If you want to use `RobertaLMHeadModel` as a standalone, add `is_decoder=True.`


## Attack 

In [ ]:
hf_dataset = HuggingFaceDataset(attack_recipes.ds.dsd_raw[param_d['split']], dataset_columns=(['text'], 'label'))
for attack_json in attack_list:
    print("Now doing attack recipe number", attack_json['attack_num'], "with code", attack_json['attack_code'])
    datetime_now = datetime.now().strftime("%Y-%m-%d_%H%M%S")
    param_d['datetime'] =  datetime_now
    param_d['attack_num'] = attack_json['attack_num']
    param_d['attack_code'] = attack_json['attack_code']
    filename = f"{path_baselines}{datetime_now}_{param_d['ds_name']}_{param_d['split']}_{attack_json['attack_code']}.csv"
    attack_args = AttackArgs(num_examples=-1, enable_advance_metrics=True,
                            log_to_csv=filename, csv_coloring_style='plain', disable_stdout=True)
    attacker = Attacker(attack_json['attack_recipe'], hf_dataset, attack_args)

    # print("Current config for attack:")
    # print(d)

    attack_results = attacker.attack_dataset()

    attack_result_metrics = {
        **AttackSuccessRate().calculate(attack_results), 
        **WordsPerturbed().calculate(attack_results),
        **AttackQueries().calculate(attack_results),
        **Perplexity().calculate(attack_results),
        **USEMetric().calculate(attack_results)
    }
    attack_result_metrics.pop('num_words_changed_until_success')
    d = merge_dicts(param_d, attack_result_metrics)
    summary_df = pd.Series(d).to_frame().T
    append_df_to_csv(summary_df, f"{path_baselines}results.csv")

textattack: Logging to CSV at path ./baselines/2022-07-04_145655_financial_test_LM-WR-BS-b2m5.csv
  0%|          | 0/3 [00:00<?, ?it/s]

Attack(
  (search_method): BeamSearch(
    (beam_width):  2
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapMaskedLM(
    (method):  bae
    (masked_lm_name):  RobertaForCausalLM
    (max_length):  512
    (max_candidates):  5
    (min_confidence):  0.0005
  )
  (constraints): 
    (0): StsScoreConstraint(
        (compare_against_original):  True
      )
    (1): ContradictionScoreConstraint(
        (compare_against_original):  True
      )
    (2): AcceptabilityScoreConstraint(
        (compare_against_original):  True
      )
    (3): PpLetterDiffConstraint(
        (compare_against_original):  True
      )
    (4): LCPConstraint(
        (compare_against_original):  True
      )
    (5): RepeatModification
    (6): StopwordModification
  (is_black_box):  True
) 



 33%|███▎      | 1/3 [00:13<00:26, 13.19s/it]/home/tproth/Programs/miniconda/envs/nlp_env/lib/python3.8/site-packages/textattack/loggers/csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:  67%|██████▋   | 2/3 [00:18<00:09,  9.44s/it]/home/tproth/Programs/miniconda/envs/nlp_env/lib/python3.8/site-packages/textattack/loggers/csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
[Succeeded / Failed / Skipped / Total] 1 / 1 / 0 / 2: 100%|██████████| 3/3 [00:22<00:00,  7.64s/it]/home/tproth/Programs/miniconda/envs/nlp_env/lib/python3.8/site-packages/textattack/loggers/csv_logger.py:41: FutureWarning: The frame.append method is deprecated and wi


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 2      |
| Number of failed attacks:     | 1      |
| Number of skipped attacks:    | 0      |
| Original accuracy:            | 100.0% |
| Accuracy under attack:        | 33.33% |
| Attack success rate:          | 66.67% |
| Average perturbed word %:     | 26.24% |
| Average num. words per input: | 16.0   |
| Avg num queries:              | 80.33  |
| Average Original Perplexity:  | 573.64 |
| Average Attack Perplexity:    | 761.11 |
| Average Attack USE Score:     | 0.71   |
+-------------------------------+--------+


textattack: Logging to CSV at path ./baselines/2022-07-04_145745_financial_test_LM-WR-BS-b5m25.csv
  0%|          | 0/3 [00:00<?, ?it/s]

...


+-------------------------------+---------+
| Attack Results                |         |
+-------------------------------+---------+
| Number of successful attacks: | 2       |
| Number of failed attacks:     | 1       |
| Number of skipped attacks:    | 0       |
| Original accuracy:            | 100.0%  |
| Accuracy under attack:        | 33.33%  |
| Attack success rate:          | 66.67%  |
| Average perturbed word %:     | 13.57%  |
| Average num. words per input: | 16.0    |
| Avg num queries:              | 362.0   |
| Average Original Perplexity:  | 573.64  |
| Average Attack Perplexity:    | 1137.85 |
| Average Attack USE Score:     | 0.83    |
+-------------------------------+---------+


textattack: Logging to CSV at path ./baselines/2022-07-04_151119_financial_test_LM-WR-GA-m25p60mi20mr5.csv
  0%|          | 0/3 [00:00<?, ?it/s]

Attack(
  (search_method): ImprovedGeneticAlgorithm(
    (pop_size):  60
    (max_iters):  20
    (temp):  0.3
    (give_up_if_no_improvement):  False
    (post_crossover_check):  False
    (max_crossover_retries):  20
    (max_replace_times_per_index):  5
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapMaskedLM(
    (method):  bae
    (masked_lm_name):  RobertaForCausalLM
    (max_length):  512
    (max_candidates):  25
    (min_confidence):  0.0005
  )
  (constraints): 
    (0): StsScoreConstraint(
        (compare_against_original):  True
      )
    (1): ContradictionScoreConstraint(
        (compare_against_original):  True
      )
    (2): AcceptabilityScoreConstraint(
        (compare_against_original):  True
      )
    (3): PpLetterDiffConstraint(
        (compare_against_original):  True
      )
    (4): LCPConstraint(
        (compare_against_original):  True
      )
    (5): RepeatModification
    (6): StopwordModification
  (is_black_box):  Tr

 33%|███▎      | 1/3 [00:28<00:56, 28.31s/it]/home/tproth/Programs/miniconda/envs/nlp_env/lib/python3.8/site-packages/textattack/loggers/csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:  67%|██████▋   | 2/3 [03:10<01:35, 95.10s/it]/home/tproth/Programs/miniconda/envs/nlp_env/lib/python3.8/site-packages/textattack/loggers/csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
[Succeeded / Failed / Skipped / Total] 1 / 1 / 0 / 2: 100%|██████████| 3/3 [03:32<00:00, 70.90s/it]/home/tproth/Programs/miniconda/envs/nlp_env/lib/python3.8/site-packages/textattack/loggers/csv_logger.py:41: FutureWarning: The frame.append method is deprecated and wi


+-------------------------------+---------+
| Attack Results                |         |
+-------------------------------+---------+
| Number of successful attacks: | 2       |
| Number of failed attacks:     | 1       |
| Number of skipped attacks:    | 0       |
| Original accuracy:            | 100.0%  |
| Accuracy under attack:        | 33.33%  |
| Attack success rate:          | 66.67%  |
| Average perturbed word %:     | 20.36%  |
| Average num. words per input: | 16.0    |
| Avg num queries:              | 917.67  |
| Average Original Perplexity:  | 573.64  |
| Average Attack Perplexity:    | 1080.47 |
| Average Attack USE Score:     | 0.75    |
+-------------------------------+---------+


## Example-specific metrics 

In [ ]:
def display_adv_example(df): 
    from IPython.core.display import display, HTML
    pd.options.display.max_colwidth = 480 # increase column width so we can actually read the examples
    #display(HTML(df[['original_text', 'perturbed_text']].to_html(escape=False)))
    display(df[['original_text', 'perturbed_text']])

# def add_vm_score_and_label_flip(df, dataset, cfg, vm_tokenizer, vm_model): 
#     truelabels = torch.tensor(dataset._dataset['label'], device =cfg.device)
#     orig_probs =  get_vm_probs(df['original_text'].tolist(), cfg, vm_tokenizer, vm_model, return_predclass=False)
#     pp_probs = get_vm_probs(df['perturbed_text'].tolist(), cfg, vm_tokenizer, vm_model, return_predclass=False)
#     orig_predclass = torch.argmax(orig_probs, axis=1)
#     pp_predclass = torch.argmax(pp_probs, axis=1)
#     orig_truelabel_probs = torch.gather(orig_probs, 1, truelabels[:,None]).squeeze()
#     pp_truelabel_probs   = torch.gather(pp_probs, 1,   truelabels[:,None]).squeeze()
#     pp_predclass_probs   = torch.gather(pp_probs, 1,   pp_predclass[ :,None]).squeeze()
    
#     df['truelabel'] = truelabels.cpu().tolist()
#     df['orig_predclass'] = orig_predclass.cpu().tolist()
#     df['pp_predclass'] = pp_predclass.cpu().tolist()
#     df['orig_truelabel_probs'] = orig_truelabel_probs.cpu().tolist()
#     df['pp_truelabel_probs'] = pp_truelabel_probs.cpu().tolist()
#     df['vm_scores'] = (orig_truelabel_probs - pp_truelabel_probs).cpu().tolist()
#     df['label_flip'] = ((pp_predclass != truelabels) * 1).cpu().tolist()
#     return df

# def add_sts_score(df, sts_model, cfg): 
#     orig_embeddings  = sts_model.encode(df['original_text'].tolist(),  convert_to_tensor=True, device=cfg.device)
#     pp_embeddings    = sts_model.encode(df['perturbed_text'].tolist(), convert_to_tensor=True, device=cfg.device)
#     df['sts_scores'] = pytorch_cos_sim(orig_embeddings, pp_embeddings).diagonal().cpu().tolist()
#     return df

# def add_contradiction_score(df, cfg, nli_tokenizer, nli_model): 
#     contradiction_scores = get_nli_probs(df['original_text'].tolist(), df['perturbed_text'].tolist(), cfg, nli_tokenizer, nli_model)
#     df['contradiction_scores'] =  contradiction_scores[:,cfg.contra_label].cpu().tolist()
#     return df 

# def get_df_mean_cols(df): 
#     cols = ['label_flip', 'vm_scores', 'sts_scores',
#             'contradiction_scores', 'sts_threshold_met', 'contradiction_threshold_met']
#     s = df[cols].mean()
#     s.index = [f"{o}_mean" for o in s.index]
#     return dict(s)

# def get_cts_summary_stats(df): 
#     cols = ['vm_scores', 'sts_scores', 'contradiction_scores']
#     df_summary = df[cols].describe(percentiles=[.1,.25,.5,.75,.9]).loc[['std','10%','25%','50%','75%','90%']]
#     tmp_d = dict()
#     for c in cols: 
#         s = df_summary[c]
#         s.index = [f"{c}_{o}" for o in s.index]
#         tmp_d = merge_dicts(tmp_d, dict(s))
#     return tmp_d


In [ ]:
#filename1 = f"/data/tproth/travis_attack/baselines/2022-04-21_044443_rotten_tomatoes_valid_BeamSearchLMAttack_beam_sz=2_max_candidates=5.csv"
#filename = filename1
df = pd.read_csv(filename)
#display_adv_example(df)

#df = add_vm_score_and_label_flip(df, dataset, cfg, vm_tokenizer, vm_model)
#df = df.query("result_type != 'Skipped'")
#df = add_sts_score(df, sts_model, cfg)
#df = add_contradiction_score(df, cfg, nli_tokenizer, nli_model)

#df['sts_threshold_met'] = df['sts_scores'] > d['sts_threshold']
#df['contradiction_threshold_met'] = df['contradiction_scores'] < d['contradiction_threshold']
#df.to_csv(f"{filename[:-4]}_processed.csv", index=False)

#d = merge_dicts(d, get_df_mean_cols(df))
#d = merge_dicts(d, get_cts_summary_stats(df))



## Old code 

In [ ]:
# df1 = df.sample(5)
# orig_l = df1['original_text'].tolist()
# pp_l = df1['perturbed_text'].tolist()
# print(orig_l)
# print(pp_l)

In [ ]:
# for orig, adv in zip(df1['original_text'].tolist(), df1['perturbed_text'].tolist()): 
#     print(f"{orig}{adv}")
#     print()

In [ ]:
#df.iloc[104][['original_text', 'perturbed_text']].values

In [ ]:
#filename1 = f"/data/tproth/travis_attack/baselines/2022-04-20_133329_rotten_tomatoes_valid_BeamSearchCFEmbeddingAttack_beam_sz=1_max_candidates=1_processed.csv"
#df = pd.read_csv(filename1)
#display_all(df.sample(2))

In [ ]:
#df_results = pd.read_csv(f"/data/tproth/travis_attack/baselines/results.csv")